In [44]:
import os
# import sys
# import cv2
# import numpy as np
import pandas as pd
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# from IPython.display import clear_output

# import torch
# from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
# from torchvision.models.detection import fasterrcnn_resnet50_fpn

from ranavision import *

# from google.colab import drive
# drive.mount('/content/drive')



In [45]:
# ============================================
# ================ CONFIG ====================
# ============================================
BASE_URL_ANALISIS_FISICO = '/home/sinergia-terminal-6/Documentos/ANALISIS FISICO RIPLEY 2023/'
BASE_URL_RANAVISION = './'
BASE_URL_RESULTADOS = './'

device = 'cuda'


# crea la carpeta de resultados
if not 'output' in os.listdir( BASE_URL_RESULTADOS ):
    os.mkdir( BASE_URL_RESULTADOS + 'output' )



In [46]:
# ============================================
# ========== LOAD MODELS N STUFF =============
# ============================================


# DEVICE
device = 'cuda'         # cuda o cpu ; cuda es gpu

# MODELO SEGMENTACION
weights = torchvision.models.segmentation.FCN_ResNet50_Weights.DEFAULT
preprocess = weights.transforms()
model_seg = torchvision.models.segmentation.fcn_resnet50(num_classes=2).to(device)
model_seg.load_state_dict(torch.load('ranavision_segmentation.pt', map_location=device))
model_seg = model_seg.eval()

# MODELO DETECCION
model_det = torchvision.models.detection.fasterrcnn_resnet50_fpn(num_classes=11).to(device)
model_det.load_state_dict(torch.load('ranavision_detection.pt', map_location=device))
model_det = model_det.eval()
pass


In [ ]:
# ============================================
# ============= RECORRER CSVS ================
# ============================================


for folder in os.listdir(BASE_URL_ANALISIS_FISICO):                             # recorre las carpetas dentro del analisis fisico
  subfolders = os.listdir(BASE_URL_ANALISIS_FISICO + folder)
  if 'CSVs' in subfolders:
    for filename in os.listdir(BASE_URL_ANALISIS_FISICO + folder + '/CSVs'):    # recorre todos los archivos dentro de la carpetas CSVs
      if 'csv' in filename:
        # lee el csv
        try:
          csv = pd.read_csv(BASE_URL_ANALISIS_FISICO + folder + '/CSVs/' + filename, delimiter='|')
          print('Leyendo',folder,filename)
        except Exception as e:
          print(e)
          print('Error procesando archivo:',folder + '/CSVs/'+filename)
          continue
        
        # verifica que tenga las columnas indicadas
        if (not 'NOMBRE IMAGEN PARTE' in csv.keys()) or (not 'SKU' in csv.keys() or not 'EAN' in csv.keys()) or (not 'DESCRIPCION' in csv.keys()):
          print(filename,'no tiene los nombres de campos correctos')
          continue

        # verifica cada imagen
        im_names = csv['NOMBRE IMAGEN PARTE'].to_list()
        pesos = csv['PESO INFORMADO'].to_list()
        verified = list()
        for imname,peso in zip(im_names,pesos):
          try:
              im = cv2.imread(  BASE_URL_ANALISIS_FISICO + folder +'/Images/' + imname)
              pantalla = getPantalla(im,model_seg,preprocess,device)          # segmentar la pantalla
              pred = predictDigitos(pantalla,model_det,device,0)            # generar la prediccion
              if type(peso) == str:
                peso = float( peso.replace(',','.'))
              iguales = (compareDigitos(pred,peso*100,verbose=0) or compareDigitos(pred,peso*1000,verbose=0)) > 0
              verified.append( iguales )
          except Exception as e:
              # print(e)
              verified.append(False)

        # guarda el archivo reporte
        resultado = pd.DataFrame(data={'SKU':csv['SKU'],
                                        'EAN':csv['EAN'],
                                        'DESCRIPCION':csv['DESCRIPCION'], 
                                        'Peso':pesos,
                                        'Imagename':im_names,
                                        'Verified':verified})
        resultado.to_csv(BASE_URL_RESULTADOS + f'output/{folder}_{filename}')

In [63]:
for filename in os.listdir('output'):
    csv = pd.read_csv(f'output/{filename}')
    true = csv['Verified'].value_counts()[True]
    total = len(csv)
    print(filename,f'{true}/{total}',round(true/total,2))
    

SINERGIA-NOTE-07 (ALEJANDRO URBINA)_AnalisisFisico.csv 115/325 0.35
SINERGIA-TAB-06 (GONZALO ARANCIBIA)_AnalisisFisico CORRECCION 2FEB2024.csv 288/577 0.5
SINERGIA-NOTE-19 (RICARDO OLIVARES)_AnalisisFisico.csv 419/778 0.54
SINERGIA-NOTE-17 (ALEJANDRO URBINA)_AnalisisFisico.csv 201/507 0.4
SINERGIA-TAB-06 (GONZALO ARANCIBIA)_AnalisisFisico CORRECCION 1FEB2024.csv 171/575 0.3
